<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch12_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

# 사용자 정의 모델
- keras.Model 클래스를 상속
- 생성자에서 층과 변수를 만든다
- call()메서드에 모델이 해야 할 작업을 구현한다.

## 스킵 연결이 있는 사용자 정의 잔차 블록(ResidualBlock) 층을 가진 모델 만들어보기

### Residual Block

In [ ]:
class ResidualBlock(keras.layers.Layer): #1. Layer클래스 상속
  def __init__(self, n_layers, n_neurons, **kwargs): #2. 필요한 층을 만들고
    super().__init__(**kwargs)
    self.hidden=[keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal') for _ in range(n_layers)] #n_layers개의 Dense layer

  def call(self, inputs):#3. call메서드에 구현한다.
    Z=inputs
    for layer in self.hidden:
      Z=layer(Z)
    return inputs+Z #n_layers개의 은닉층을 거친 결과인 Z에 inputs값을 더해주기(skip connection)

### Residual Regressor

In [ ]:
class ResidualRegressor(keras.Model):#1. Model 클래스 상속
  def __init__(self, output_dim, **kwargs): #2. 필요한 층을 만들고
    super().__init__(**kwargs)
    self.hidden1=keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal')
    self.block1=ResidualBlock(2,30) #30개의 유닛을 가진 hidden layer 2개 거친 후 skip connection
    self.block2=ResidualBlock(2,30) #두 번째 잔차 블럭
    self.out=keras.layers.Dense(output_dim)

  def call(self,inputs): #3. call메서드에 구현
    Z=self.hidden1(inputs)
    for _ in range(3):#Residual block 여러 번 통과시키기
      Z=self.block1(Z)
    Z=self.block2(Z)
    return self.out(Z)


----------------
- Model 클래스는 Layer 클래스의 서브클래스이므로 모델을 층처럼 정의할 수 있다.
- 하지만 모델은 compile(), fit(), evaluate(), predict(), save() 등의 메서드와 같은 추가적인 기능이 있다.


Q) Model이 더 많은 기능을 제공한다면, 모든 Layer를 Model처럼 정의하면 되지 않나?<br>
A) 모든 층을 모델처럼 정의할 수 있지만, <strong>모델 안의 내부 구성 요소(재사용 가능한 층의 블럭 등)</strong>를 <strong>모델(훈련 대상 객체)과 구분</strong>하는 것이 당연하므로 Layer, Model 따로 상속해서 구현하는 것이다.


## 모델 구성 요소에 기반한 손실과 지표

- 모델 구성 요소(은닉층의 가중치나 활성화 함수 등)에 기반한 손실을 정의하고 계산한다.
- add_loss()메서드에 계산 결과를 전달한다.

### 사용자 정의 재구성 손실을 가지는 모델 만들기
- 다섯 개의 은닉층과 출력층으로 구성된 회귀 MLP모델
- 해당 모델은 맨 위의 은닉층에 보조 출력을 가진다. => 이 보조 출력에 연결된 손실이 재구성 손실
- 재구성 손실: 보조 출력 값(재구성)과 input 값 사이의 오차(보통 오토인코더에 사용)

In [ ]:
class ReconstructingRegressor(keras.Model):
  def __init__(self, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.hidden=[keras.layers.Dense(30, activation='selu', kernel_initializer='lecun_normal') for _ in range(5)]
    self.out=keras.layers.Dense(output_dim)

  def build(self, batch_input_shape): #Dense layer를 더 추가해서 모델의 입력을 재구성하는 데 사용한다.
    n_inputs=batch_input_shape[-1] #유닛 개수는 입력 개수와 같아야 한다.
    self.reconstruct=keras.layers.Dense(n_inputs)
    super().build(batch_input_shape)

  def call(self, inputs):
    Z=inputs
    for layer in self.hidden:
      Z=layer(Z)
    reconstruction=self.reconstruct(Z) #마지막 은닉층의 보조 출력 값
    recon_loss=tf.reduce_mean(tf.square(reconstruction-inputs)) #재구성 손실: 보조 출력 값(재구성)과 input 값 사이의 평균 제곱 오차
    self.add_loss(0.05*recon_loss) #모델의 손실 리스트에 추가한다.
    return self.out(Z)

------------
 0.05(하이퍼파라미터)를 곱하는 이유는, 재구성 손실이 주 손실을 압도하지 않도록 하기 위해서다. (그냥 규제처럼 작용하기 위함)

## 자동 미분을 사용하여 Gradient 계산하기

In [ ]:
def f(w1,w2):
  return 3*w1**2+2*w1*w2

--------------
- w1에 대한 함수 f의 도함수는 6*w1+2*w2
- w2에 대한 함수 f의 도함수는 2*w1
- (w1,w2) = (5, 3)에서 도함수의 값은 각각 36, 10 -> gradient vector: (36,10)

-> 신경망은 보통 수만 개의 파라미터를 가지므로 이를 직접 다 계산하는 것은 불가능하다.<br>
-> <strong>대안: 각 파라미터가 바뀔 때마다 함수의 출력이 얼마나 변하는지 측정해서 도함수의 근삿값을 계산하는 것</strong>

In [ ]:
w1,w2=5,3
eps=1e-6
(f(w1+eps,w2)-f(w1,w2))/eps

36.000003007075065

In [ ]:
(f(w1,w2+eps)-f(w1,w2))/eps

10.000000003174137

------------
하지만 파라미터마다 적어도 한번씩 f()를 호출한다. 따라서 대규모 신경망엔 적용하기 어렵다.<br>
<strong>-> 대안: 자동 미분을 사용해보자</strong>

### GradientTape
- 여러 값(보통 모델의 파라미터)에 대한 한 값(보통 손실값)의 gradient를 계산하는 데 사용
- 한 번의 정방향 계산과 역방향 계산으로 모든 gradient를 동시에 계산할 수 있다.(후진 모드 자동 미분)

In [ ]:
w1,w2=tf.Variable(5.), tf.Variable(3.) #먼저 두 변수 w1,w2를 정의한다.
with tf.GradientTape() as tape: # tf.GradientTape 블럭을 만들어서 두 변수와 관련된 모든 연산을 자동으로 기록한다.
  z=f(w1,w2)
gradients=tape.gradient(z,[w1,w2]) #tape에 두 변수 [w1,w2]에 대한 z의 gradient를 요청한다.

In [ ]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [ ]:
tape.gradients(z,[w1,w2]) #한 번 더 호출하면 error

AttributeError: ignored

--------------
- gradient()메서드가 호출된 후에는 자동으로 테이프가 즉시 지워진다. -> 그래서 error 발생
- 만약 한 번 이상 gradient()메서드를 호출해야한다면 persistent 값을 True로 주면 된다.

In [ ]:
with tf.GradientTape(persistent=True) as tape: #persistent=True
  z=f(w1,w2)
gradients=tape.gradient(z,[w1,w2])
tape.gradient(z,[w1,w2]) #한 번 더 호출해도 예외 발생 X

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [ ]:
del tape #단, 테이프를 삭제해서 리소스를 해제해야한다. (tape 객체가 더 이상 유효하지 않게 되면 파이썬 GC가 삭제해준다.)

### Detail
- 기본적으로 tape는 변수가 포함된 연산만을 기록한다.

#### 만약 변수가 아니라면?

In [ ]:
c1,c2=tf.constant(5.), tf.constant(3.) #상수
with tf.GradientTape() as tape:
  z=f(c1,c2)
tape.gradient(z,[c1,c2])

[None, None]

-------------
계산이 안된다.

In [ ]:
with tf.GradientTape() as tape:
  tape.watch(c1) 
  tape.watch(c2)
  z=f(c1,c2)
tape.gradient(z,[c1,c2])

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

---------------
하지만 watch()메서드로 어떤 텐서라도 감시해서 모든 연산을 기록할 수 있도록 강제할 수 있다.

#### 신경망의 일부분에 gradient가 역전파되지 않도록 막고 싶다면?
- stop_gradient(): 역전파 시에 상수처럼 동작한다.

In [ ]:
def f(w1,w2):
  return 3*w1**2+tf.stop_gradient(2*w1*w2) #역전파 시, 2*w1*w2를 상수 취급한다.

with tf.GradientTape() as tape:
  z=f(w1,w2)

tape.gradient(z,[w1,w2])

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

#### 수치적인 이슈
- 큰 입력에 대한 my_softplus()함수의 gradient를 계산하면 NaN이 반환된다.

In [ ]:
def my_softplus(z):
  return tf.math.log(tf.exp(z)+1.0)

In [ ]:
tf.exp(100.) #tf.exp는 입력값이 크면 np.inf를 반환한다.

<tf.Tensor: shape=(), dtype=float32, numpy=inf>

In [ ]:
x=tf.Variable([100.])
with tf.GradientTape() as tape:
  z=my_softplus(x)
tape.gradient(z,[x])

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

---------
- 부동소수점 정밀도 오류로 인해서 자동 미분이 무한 나누기 무한을 하게 된다. -> 그래서 NaN 반환

@tf.custom_gradient 데코레이터로 수치적으로 안전한 softplus의 도함수를 계산해서 반환하는 함수 작성하기

In [ ]:
@tf.custom_gradient
def my_better_softplus(z):
  exp=tf.exp(z)

  def my_softplus_gradients(grad):
    return grad/(1+1/exp) #exp가 np.inf면 -> grad/(1+0)이 된다
  
  return tf.math.log(exp+1), my_softplus_gradients()

## 사용자 정의 훈련 반복
- fit()메서드의 유연성이 충분하지 않는 경우
- 의도한대로 잘 동작하는지 확신을 갖고 싶은 경우
- 위와 같은 경우 직접 훈련 반복을 만들 수 있다.

### 간단한 모델 만들기
- 훈련 반복을 직접 다루기에 컴파일할 필요가 없다

In [ ]:
l2_reg=keras.regularizers.l2(0.05)
model=keras.models.Sequential([
                               keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2_reg),
                               keras.layers.Dense(1,kernel_regularizer=l2_reg)

])

In [ ]:
def random_batch(X,y,batch_size=32): #샘플 배치를 랜덤하게 추출하는 함수
  idx=np.random.randint(len(X), size=batch_size)
  return X[idx], y[idx]

In [ ]:
def print_status_bar(iteration, total, loss, metrics=None): #훈련 상태 출력 함수
  metrics=" - ".join([f"{m.name}:{m.result():.4f}" for m in [loss]+(metrics or [])])
  end="" if iteration < total else "\n"
  print(f"\r{iteration}/{total}" + metrics, end=end)

### 적용해보기

In [ ]:
X_train=tf.constant([1,2,3,4,5])
y_train=tf.constant([3,4,5,6,7])

In [ ]:
n_epochs=5
batch_size=32
n_steps=len(X_train)//batch_size
optimizer=keras.optimizers.Nadam(lr=0.01)
loss_fn=keras.losses.mean_squared_error
mean_loss=keras.metrics.Mean()
metrics=[keras.metrics.MeanAbsoluteError()]

In [ ]:
for epoch in range(1,n_epochs+1): #epoch
  print(f"epoch {epoch}/{n_epochs}")
  for step in range(1,n_steps+1): #batch
    X_batch, y_batch = random_batch(X_train_scaled, y_train) #랜덤하게 샘플링
    with tf.GradientTape() as tape: 
      y_pred =model(X_batch, training=True) #배치 하나를 위한 예측을 만들고
      main_loss=tf.reduce_mean(loss_fn(y_batch, y_pred)) #손실을 계산하고 모든 샘플에 대한 평균 손실을 구한다
      loss=tf.add_n([main_loss]+model.losses) #손실을 모두 더한다

    gradients=tape.gradient(loss, model.trainable_variables) #훈련 가능한 변수에 대한 손실의 gradient 계산
    optimizer.apply_gradients(zip(gradients, model.trainable_variables)) #gradient descent 수행
    mean_loss(loss)
    for metric in metrics:
      metric(y_batch, y_pred)
    print_status_bar(step*batch_size, len(y_train), mean_loss, metrics)
  print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
  for metric in [mean_loss] + metrics:
    metric.reset_states()


epoch 1/5
5/5mean:0.0000 - mean_absolute_error:0.0000
epoch 2/5
5/5mean:0.0000 - mean_absolute_error:0.0000
epoch 3/5
5/5mean:0.0000 - mean_absolute_error:0.0000
epoch 4/5
5/5mean:0.0000 - mean_absolute_error:0.0000
epoch 5/5
5/5mean:0.0000 - mean_absolute_error:0.0000


---------
- 이 처럼 사용자 정의 훈련 반복 구현은 주의해야 할 점이 많고, 실수하기 쉽다.
- 장점은 완전히 제어할 수 있다는 것

## 텐서플로 함수와 그래프
- 텐서플로 2에서는 1보다 훨씬 쉽게 그래프를 사용하기 쉽다.
- 텐서플로 함수는 파이썬 함수보다 훨씬 빠르게 실행한다.(복잡한 연산을 수행할 때 더욱 두드러진다)

### 일반 파이썬 함수 -> 텐서플로 함수

#### 파이썬 함수

In [ ]:
def cube(x):
  return x**3

In [ ]:
cube(2)

8

In [ ]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

#### 텐서플로 함수로 변환
- 내부적으로 tf.function은 cube()함수에서 수행되는 계산을 분석하고 동일한 작업을 수행하는 계산 그래프를 생성한다.

In [ ]:
#방법 1: 함수 적용
tf_cube=tf.function(cube)
tf_cube 

In [ ]:
#방법 2: 데코레이터
@tf.function
def tf_cube(x):
  return x**3

In [ ]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [ ]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

-----------
파이썬 함수 -> 텐서플로 함수로 변환 후, 텐서를 반환한다.
- 주의: 파이썬 값으로 텐서플로 함수를 여러번 호출하면 프로그램이 느려진다.(매번 새로운 그래프를 생성해서)


## 텐서플로가 그래프를 생성하는 방법은?
<img src="https://pic1.zhimg.com/80/v2-be09ffe32f08eab81e78edec00f681b8_1440w.jpg" width=70% height=70%/>

1. 파이썬 함수의 소스 코드를 분석해서 for, while, if, break 등 제어문을 모두 찾는다. -> 이 단계를 "오토그래프"라고 한다.
2. 1에서 찾은 제어문을 텐서플로 연산으로 바꾼 새로운 버전의 함수를 만든다.
3. 텐서플로가 2에서 만들어진 함수를 호출한다. -> 해당 함수는 "그래프 모드"로 실행된다. 
4. 최종 그래프는 "트레이싱"과정을 통해 생성된다
